## This notebook ingest the raw "constructors" file from the bucket to the bronze layer

In [1]:
# Define parameters (can set parameters in a workflow job)
target_type   =oidlUtils.parameters.getParameter("TARGET_TYPE", "table")
target_format =oidlUtils.parameters.getParameter("TARGET_FORMAT", "delta")
bronze_folder_path    = "/Volumes/f1_bronze/bronze/f1_bronze_volume"
bronze_catalog    = "f1_bronze"
bronze_schema     = "bronze"
bronze_table_dlt = "f1_constructors_dlt"
bronze_table_par = "f1_constructors_par"

# ----------
##### Step 1 - Read the CSV file using the spark dataframe reader
#  ----------

In [1]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DateType

In [1]:
constructors_schema = StructType(fields=[StructField("constructorId", IntegerType(), False),
                                    StructField("constructorRef", StringType(), True),
                                    StructField("name", StringType(), True),
                                    StructField("nationality", StringType(), True),
                                     StructField("url", StringType(), True)
])

In [1]:
constructor_df = spark.read \
.option("header", True) \
.schema(constructors_schema) \
.csv(f"{bronze_folder_path}/constructors.csv")

# ----------
##### Step 2 Write output in selected format
#  ----------

In [1]:
if target_type == 'file':
    if target_format == 'parquet':
        constructor_df.write.mode("overwrite").parquet(f"{bronze_folder_path}/constructors_par")
elif target_type == 'table':
    if target_format == 'parquet':
        constructor_df.write.mode("overwrite").format("parquet").saveAsTable(f"{bronze_catalog}.{bronze_schema}.{bronze_table_par}")
    elif  target_format == 'delta':
        constructor_df.write.mode("overwrite").format("delta").saveAsTable(f"{bronze_catalog}.{bronze_schema}.{bronze_table_dlt}")

In [1]:
if target_type == 'file':
    if target_format == 'parquet':
        constructor_read_df = spark.read.parquet(f"{bronze_folder_path}/constructors_par")
elif target_type == 'table':
    if target_format == 'parquet':
        constructor_read_df = spark.read.table(f"{bronze_catalog}.{bronze_schema}.{bronze_table_par}")
    elif  target_format == 'delta':
        constructor_read_df = spark.read.table(f"{bronze_catalog}.{bronze_schema}.{bronze_table_dlt}")